In [686]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import OrderedDict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [687]:
from csv_data import get_vocab_dict
from csv_data import get_all_articles
from csv_data import csv_list_to_list
from csv_data import split_data
from csv_data import check_duplicates


# # TODO : fjern ord der er kategorisert som "egennavn" i ddo_fullforms_2020-08-26.csv

ordered_dict = get_vocab_dict()
data_total, labels_total = get_all_articles(shuffle=True)
nationalities = csv_list_to_list('resources/nat2.csv')
danske_navne = csv_list_to_list('resources/danmark_navne.csv')
andre_navne = csv_list_to_list('resources/andre_navne.csv')
countries = csv_list_to_list('resources/countries.csv')
navne = csv_list_to_list('resources/navne.csv')

check_duplicates(data_total)

train_data, val_data, train_labels, val_labels = split_data((data_total, labels_total), 5)

print("Train data/labels length: ", len(train_data), len(train_labels))
print("Validation data/labels length: ", len(val_data),  len(val_labels))


Train data/labels length:  8521 8521
Validation data/labels length:  448 448


In [688]:
import re

for i,tt in enumerate(data_total):
    if re.search(r'\btaber', tt, re.IGNORECASE):
        if (labels_total[i] == 0):
            print(tt)
            print(labels_total[i])
        # print(train)



amerikansk fodbold . kelce-brødre kæmper om håneretten i super bowl . travis og jason kelce er brødre og skal møde hinanden i super bowl. én sikrer triumfen, mens den anden går hjem som taber
0
ishockey . dansk ishockeyformand taber valg til præsidentpost . henrik bach nielsen blev ikke valgt som afløser for iihf's mangeårige præsident, rené fasel
0
kort sport  . dansk golfturnering taber kampen mod vinden  . blæsevejret på møn er blevet så kraftigt, at man ikke længere kan spille golf i det
0
formel 1 . kevin magnussen taber to kilo i sydfransk svedekasse . kevin magnussen leverede fredag en fin anden træning op til et fransk grandprix, der ligner en hed omgang
0
amerikansk fodbold . her taber spiller hovedet med uhørt handling . for første gang i 23 år modtog en spiller to bortvisninger i samme sæson. den ikke så eftertragtede bedrift står førsteårsspilleren quay walker bag
0
vm fodbold . hollandsk træner deler vandene - snart kan det være slut . louis van gaal skal fredag forsøge at

In [689]:
counts = np.bincount(train_labels)
# print(
#     "Number of positive samples in training data: {} ({:.2f}% of total)".format(
#         counts[1], 100 * float(counts[1]) / len(train_labels)
#     )
# )

print(counts)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

class_weight = {0: weight_for_0, 1: weight_for_1}

[4778 3743]


In [690]:

# def combine_articles_to_csv():
#     df_sport_combined = df_sport.copy().drop('Link', axis=1)
#     df_sport_combined.to_csv('articles_temp/combined.csv')

# combine_articles_to_csv()


In [691]:
# duplicate_rows = df_sport.duplicated()
# print("Duplicates in data points: ")
# print(df_sport[duplicate_rows])

In [692]:
import time
isin_dict = False
def test_lookup_performance():
    word_to_check = "trænerfronten"
    start_time = time.time()

    for x in range(1000000):
        isin_dict = word_to_check in ordered_dict

    end_time = time.time()  
    assert(end_time - start_time < 1)
    print(isin_dict)

test_lookup_performance()

# isin_dict


True


In [693]:


# train_text = df_sport.iloc[:, [0,1,2]].apply(' . '.join, axis=1).replace('\xa0', '', regex=True).to_numpy()
# train_text_results = df_sport.loc[df_sport['isResult'] == True].iloc[:, [0,1,2]].apply(' . '.join, axis=1).replace('\xa0', '', regex=True).to_numpy()

from create_vocab import split_sentences
from create_vocab import remove_duplicates
from create_vocab import remove_nationalities
from create_vocab import remove_danske_navne
from create_vocab import remove_danske_fornavne
from utils import remove_words_containing_digits
from create_vocab import remove_non_dict_words
from create_vocab import add_non_alpha_numeric

# words_total =  train_data.copy()
# words_total.extend(val_data)

def words_by_frequency(arr):
    xx = np.array(arr, dtype=object)
    unique, counts = np.unique(xx, return_counts=True)
    aa = np.asarray((unique, counts)).T
    return np.flip(aa[aa[:, 1].argsort()])

def remove_non_frequent(words_arr, threshold):
    words_dict = words_by_frequency(words_arr)
    words_above_threshold = []
    for f in words_dict:
        if f[0] > threshold:
            words_above_threshold.append(f[1])
    return words_above_threshold


sentence_words = split_sentences(train_data)
sentence_words_frequent = remove_non_frequent(sentence_words, 1)
words_arr_unique = remove_duplicates(sentence_words_frequent)
words_arr_unique = remove_nationalities(words_arr_unique, nationalities)
words_arr_unique = remove_danske_navne(words_arr_unique, danske_navne)
words_arr_unique = remove_danske_fornavne(words_arr_unique, andre_navne)
words_arr_unique = remove_words_containing_digits(words_arr_unique)


words_train_vocab, words_sport_lingo = remove_non_dict_words(words_arr_unique, ordered_dict)


# # TODO : brug tensorflow Tokenezier til at omdanne ord til tokens
# # TODO : søg i alle leksikoner, søg med og uden bindestreg
# # TODO : håndter tal ikke i ordbøger eks ( x-x eller x-årig)
# # TODO : lemmatizer : udelad bøjninger af samme navneord. eks : verdensmester/verdensmesteren
# # TODO : evt. grupper ord der ofte hænger sammen med nltk BigramFinder. eks vandt over
# TODO : fjern evt. også alle navne (fornavne og efternavne)  

print("total unique words:", len(words_arr_unique) )
print("total sports lingo words:", len(words_sport_lingo) )
print("total vocab:", len(words_train_vocab))
# print("total articles:", len(df_sport) )

# for d in df_sport['isResult']:
#     if isinstance(d, bool) != True:
#         print(d)



['signe', 'trine', 'groves', 'real', 'tiger', 'rune', 'søren', 'kim', 'norman', 'mads', 'o', 'mark']
total unique words: 9275
total sports lingo words: 2222
total vocab: 7053


In [694]:

len(words_train_vocab)

7053

In [695]:
file = open('words_sport_lingo.txt','w')
for item in words_sport_lingo:
	file.write(item+"\n")
file.close()

file = open('words_train_vocab.txt','w')
for item in sorted(words_train_vocab):
	file.write(item+"\n")
file.close()


In [696]:


# TODO : lav en negativ liste også
# display most frequent words found in lingo words
for f in remove_duplicates(sentence_words):
    if f in words_sport_lingo and f not in navne and len(f) > 1:
        print(f)


hammarby
cken


In [697]:


# TODO : lav en negativ liste også
# display most frequent words found in lingo words
# words_that_appear_once = []
# for f in frequent_words:
#     if f[0] < 2 and f[1] not in navne:
#         words_that_appear_once.append(f[1])

# # print(len(words_that_appear_once))
# sorted(words_that_appear_once)


In [698]:


# f = frequent_words[-300:]
# ff = []
# for w in frequent_words:
#     if w[1] in words_train_vocab and w[0] < 2:
#         ff.append(w[1])

# print(len(ff))
# ff
    

In [699]:


from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from vectorization import replace_finals, replace_nationality, to_lower
from vectorization import split_dash
from vectorization import split_included_specials
from vectorization import replace_tournament
from vectorization import replace_countries
from vectorization import replace_weekday
from vectorization import replace_digits
from vectorization import vect_layer_2_text
from vectorization import vectorize_layer
from vectorization import standardize
from static_data import tournaments
from static_data import weekdays
from static_data import non_alpha
from static_data import word_generalization


# TODO : evt indikere hvilke navneord der starte med stort bogstav(egenavne), evt. lave et opslag for at undersøge ordklasse for det første ord i sætningen 
# TODO : test hvilke standarization funktioner giver bedre resultater 

arrs = [
    to_lower, 
    split_dash, 
    split_included_specials, 
    replace_tournament(tournaments),
    replace_countries(countries), 
    replace_weekday(weekdays), 
    replace_finals,
    replace_nationality(nationalities),
    replace_digits
]

s = standardize(arrs)

words_train_vocab.extend(word_generalization)
words_train_vocab.extend(non_alpha)

# Model constants.
max_features = 7100
sequence_length = 60

vectorized_layer = vectorize_layer(max_features, sequence_length, s)

text_ds = vectorized_layer.adapt(words_train_vocab)
vect_vocab = vectorized_layer.get_vocabulary()

print("Total vocab/max_features : ",  len(vect_vocab))

print (vect_layer_2_text(vectorized_layer(["Tidligere Formel 1-kører vinder PL-guld ,Den tidligere Formel 1-kører Alex Zanardi kørte galt i 2001 og mistede begge sine ben. I dag vandt han guld ved PL i håndcykel."]), vect_vocab))



Total vocab/max_features :  6995
['tidligere' 'formel' 'xnumber' 'kører' 'vinder' '[UNK]' 'guld' 'den'
 'tidligere' 'formel' 'xnumber' 'kører' '[UNK]' '[UNK]' 'kørte' 'galt' 'i'
 'xyear' 'og' 'mistede' 'begge' 'sine' 'ben' '.' 'i' 'dag' 'vandt' 'han'
 'guld' 'ved' '[UNK]' 'i' 'håndcykel' '.']


In [700]:
# for t in train_data[0:50]:
#     print("Original \n:", t)
#     print("Text from vectorized: \n", vect_layer_2_text(
#         vectorized_layer([t]), vect_vocab
#         ))
#     print("\n")


In [701]:
train_data_vect = vectorized_layer(train_data)
val_data_vect = vectorized_layer(val_data)


In [702]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.4):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions



In [703]:
from tensorflow.keras import layers
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [704]:
m = tf.keras.metrics.BinaryAccuracy(threshold=0.97)
m.update_state([[1], [1], [0], [0]], [[0.98], [0.98], [0], [0]])
m.result().numpy()

1.0

In [705]:
from tensorflow.keras import layers
import random as python_random

def get_cnn_model():

    embedding_dim = 48

    # A integer input for vocab indices.
    inputs = tf.keras.Input(shape=(None,), dtype="int64")

    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    x = layers.Embedding(max_features, embedding_dim)(inputs)
    # x = layers.Dropout(0.5)(x)

    # Conv1D + global max pooling
    # x = layers.Conv1D(32, 11, padding="valid", activation="relu")(x)
    # x = layers.Conv1D(128, 9, padding="valid", activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    # Conv1D + global max pooling
    x = layers.Conv1D(32, 7, padding="valid", activation="relu", strides=3)(x)
    # x = layers.Conv1D(32, 7, padding="valid", activation="relu", strides=3)(x)


    x = layers.GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    # x = layers.Dense(128, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)
    # We add a vanilla hidden layer:
    # x = layers.Dense(128, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)
    # x = layers.Dropout(0.5)(x)
    x = layers.Dropout(0.3)(x)


    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

    
    cnn_model = tf.keras.Model(inputs, predictions)

    # Compile the model with binary crossentropy loss and an adam optimizer.
    cnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return cnn_model



In [706]:
from tensorflow.keras import layers
import random as python_random


def get_transformer_model():

    embed_dim =  192 # Embedding size for each token
    num_heads = 2  # Number of attention heads
    ff_dim = 512  # Hidden layer size in feed forward network inside transformer


    # A integer input for vocab indices.
    inputs = tf.keras.Input(shape=(sequence_length,), dtype="int64")

    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    # x = layers.Embedding(max_features, embed_dim)(inputs)

    embedding_layer = TokenAndPositionEmbedding(sequence_length, max_features, embed_dim)
    x = embedding_layer(inputs)
    # x = layers.Dropout(0.5)(x)
    # x = layers.Dropout(0.2)(x)

    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    # x = layers.Dropout(0.1)(x)

    # x = transformer_block(x)



    # Conv1D + global max pooling
    # x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)
    # x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)


    x = layers.GlobalMaxPooling1D()(x)
    # x = layers.Dropout(0.5)(x)

    # We add a vanilla hidden layer:
    # x = layers.Dense(32, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)

    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)
    # x = layers.Dropout(0.5)(x)



    transformer_model = tf.keras.Model(inputs, predictions)

    # Compile the model with binary crossentropy loss and an adam optimizer.
    bn =   tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.97)
    
    
    transformer_model.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")
    return transformer_model


In [707]:

def prepare_model(name):
    if (name == "cnn"):
       return get_cnn_model()
    elif (name == "transformer"):
       return get_transformer_model()
  

def filter_max_accuracy(history, threshold = 0.95):
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    list = []
    for x in range(len(acc)):
        if (acc[x] > threshold):
            list.append(val_acc[x])

    return np.array(list)

models = ["cnn", "transformer"]

callback_3_loss = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4)


def mean_model_accuracy(mode_names, iterations, epochs = 20):

  
    results = []

    for name in range(len(mode_names)):
        model_name = mode_names[name]
        val_accuracies = []
        
        for x in range(iterations):
            model = prepare_model(model_name)

            # Fit the model using the train and test datasets.
            history = model.fit(train_data_vect, train_labels, epochs=epochs, batch_size=6, validation_data=(val_data_vect, val_labels), callbacks=[callback_3_loss])

            max_val_acc = filter_max_accuracy(history)
            val_accuracies.append(max(max_val_acc))
            print(max(max_val_acc))
            print(val_accuracies)
        
        d = dict(name = model_name, results = np.mean(np.squeeze(np.array(val_accuracies))))
        results.append(d)
        
    return results


In [708]:
# mean_results = mean_model_accuracy(models, 8)
# mean_results

In [709]:
def result_format_round(result):
    return round(result)

def result_format_none(result):
    return result

def print_model_score(model):
    score = model.evaluate(val_data_vect, val_labels, verbose=0)
    print("Validation loss:", score[0])
    print("Validations accuracy:", score[1])

def print_validation_results(predictions, val_data, labels, formatter, only_incorrect = False):
    print("Number of predictions", len(predictions))
    n_correct = 0
    for x in range(len(val_data)):
        correct_prediction = result_format_round(labels[x]) == result_format_round(predictions[x][0])
        if correct_prediction:
            n_correct += 1

        if correct_prediction == False and labels[x] == 0:
            print("VALIDATION SAMPLE TEXT: \n" ,val_data[x])
            print("VALIDATION SAMPLE DE-VECTORIZED: \n" ,vect_layer_2_text(val_data_vect[x], vect_vocab))
            print("LABEL -- :" , labels[x])
            print("PREDICTION -- :" , formatter(predictions[x][0]), " ---- float: ", predictions[x][0])
            print("CORRECT PREDICTION: ", correct_prediction)
            print("\n")

    print("Number correct: ", n_correct)

In [710]:

epochs= 6
transformer_model = get_transformer_model()

transformer_history = transformer_model.fit(train_data_vect, train_labels, epochs=epochs, batch_size=60, validation_data=(val_data_vect, val_labels),  class_weight=class_weight,)


Epoch 1/6
143/143 [==============================] - 3s 15ms/step - loss: 1.2529e-04 - accuracy: 0.7294 - val_loss: 0.4207 - val_accuracy: 0.8013
Epoch 2/6
143/143 [==============================] - 2s 15ms/step - loss: 6.7389e-05 - accuracy: 0.8789 - val_loss: 0.2374 - val_accuracy: 0.8929
Epoch 3/6
143/143 [==============================] - 2s 16ms/step - loss: 3.2688e-05 - accuracy: 0.9538 - val_loss: 0.2283 - val_accuracy: 0.8906
Epoch 4/6
143/143 [==============================] - 2s 16ms/step - loss: 1.5607e-05 - accuracy: 0.9850 - val_loss: 0.2267 - val_accuracy: 0.9107
Epoch 5/6
143/143 [==============================] - 2s 16ms/step - loss: 8.0891e-06 - accuracy: 0.9947 - val_loss: 0.2270 - val_accuracy: 0.9085
Epoch 6/6
143/143 [==============================] - 2s 16ms/step - loss: 4.2243e-06 - accuracy: 0.9987 - val_loss: 0.2312 - val_accuracy: 0.9085


In [711]:
transformer_model.summary()

Model: "model_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        [(None, 60)]              0         
_________________________________________________________________
token_and_position_embedding (None, 60, 192)           1374720   
_________________________________________________________________
transformer_block_18 (Transf (None, 60, 192)           494336    
_________________________________________________________________
global_max_pooling1d_18 (Glo (None, 192)               0         
_________________________________________________________________
predictions (Dense)          (None, 1)                 193       
Total params: 1,869,249
Trainable params: 1,869,249
Non-trainable params: 0
_________________________________________________________________


In [712]:

# epochs= 7
# cnn_model = get_cnn_model()

# transformer_history = cnn_model.fit(train_data_vect, train_labels, epochs=epochs, batch_size=40, validation_data=(val_data_vect, val_labels), class_weight=class_weight)

In [713]:
# cnn_model.summary()

In [714]:
def print_results(model):
    np.set_printoptions(precision = 5, suppress = True)
    predictions = model.predict(val_data_vect)
    print_model_score(model)
    print("\n")
    print_validation_results(predictions, val_data, val_labels, result_format_round)
  

In [715]:
print("--- TRANSFORMER ---")
print_results(transformer_model)

--- TRANSFORMER ---


Validation loss: 0.23117779195308685
Validations accuracy: 0.9084821343421936


Number of predictions 448
VALIDATION SAMPLE TEXT: 
 vm fodbold . mbappés ord i pausen af vm-finalen afsløret i dokumentar . kylian mbappé var utilfreds med frankrigs indsats i vm-finalen mod argentina og talte med store ord i pausen
VALIDATION SAMPLE DE-VECTORIZED: 
 ['xtournament' 'fodbold' '.' '[UNK]' 'ord' 'i' 'pausen' 'af' 'xtournament'
 'xfinalen' 'afsløret' 'i' 'dokumentar' '.' '[UNK]' '[UNK]' 'var'
 'utilfreds' 'med' 'xland' 'indsats' 'i' 'xtournament' 'xfinalen' 'mod'
 'xland' 'og' 'talte' 'med' 'store' 'ord' 'i' 'pausen']
LABEL -- : 0
PREDICTION -- : 1  ---- float:  0.89031833
CORRECT PREDICTION:  False


VALIDATION SAMPLE TEXT: 
 fifa vm 2022  . vinder danmark vm? modstanderne har stor respekt for det danske 'dræbergen'  . journalister fra tunesien, frankrig og australien forventer alle, at danmark når langt til vm i qatar
VALIDATION SAMPLE DE-VECTORIZED: 
 ['xtournament' 'xyear' '.' 'vinder' 'xla

In [716]:

# print("--- CNN ---")
# print_results(cnn_model)

In [717]:
import os

# Set up a logs directory, so Tensorboard knows where to look for files.

ll = transformer_model.layers[1]
ll_weights = ll.get_weights()[0]

# print(ll_weights.shape)
ll_weights


array([[ 0.00234,  0.01487, -0.02167, ...,  0.01805, -0.03029,  0.01883],
       [ 0.01066,  0.04075, -0.0136 , ..., -0.00748,  0.03641, -0.02772],
       [ 0.04616,  0.02574, -0.01313, ...,  0.03087, -0.00875,  0.02797],
       ...,
       [-0.03255, -0.00635,  0.00241, ..., -0.00908,  0.04244, -0.03505],
       [ 0.00517,  0.04537,  0.02267, ...,  0.00943, -0.01044, -0.04477],
       [-0.04426,  0.03479, -0.0473 , ...,  0.02816, -0.0376 ,  0.00743]],
      dtype=float32)

In [718]:
##import I/O module in python
import io

##open the text stream for vectors
vectors = io.open('vectors.tsv', 'w', encoding='utf-8')

##open the text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')


##write each word and its corresponding embedding
for index in range(1, len(vect_vocab)):
  word = vect_vocab[index]  # flipping the key-value in word_index
  embeddings = ll_weights[index]
  meta.write(word + "\n")
  vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")

##close the stream
vectors.close()
meta.close()

In [719]:
# from nltk import collocations
# bigram_measures = collocations.BigramAssocMeasures()
# finder = collocations.BigramCollocationFinder.from_words(["New", "York", "is", "big", "New", "York", "is", "dirty"])
# finder.ngram_fd.items()



In [720]:
# import lemmy
# # Create an instance of the standalone lemmatizer.
# lemmatizer = lemmy.load("da")

# # Find lemma for the word 'akvariernes'. First argument is an empty POS tag.
# lemmatizer.lemmatize("NOUN", "verdensetter")



In [721]:
# import nltk as nltk
# # from string import punctuation
# # from nltk.corpus import stopwords
# # nltk.download('stopwords')

# # da_stopwords = stopwords.words("danish")


In [722]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorized_layer(inputs)
# Turn vocab indices into predictions
outputs = transformer_model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)


In [723]:
print("\nResults:")


print(end_to_end_model.predict(
    [
      "Fodbold . Fjerritslev vinder over Vordingborg. Træner kommenterer på historisk kamp",
       "SPORT . Hun vandt bronze i mandags Roer Anne Dsane Andersen har som 24-årig vundet bronze ved OL",
       "Badminton . Axelsen frustreret over nederlag. Viktor Axelsen trænger til ferie efter nedturen",
      "OL . Det blev til en flot medalje til Malene dfhsds. 'Jeg er meget lykkelig for resultatet'",
      "Badminton . Dansker er videre til finalerne. dsfsdf sfdsdf bankede Fdfsdf fra Kina og skal spille i finalen på onsdag",
     ]))


print("\n NON-Results:") 
print(end_to_end_model.predict(
    [
      "Badminton . Dansker er videre til finalerne. dsfsdf sfdsdf skal spille i finalen på onsdag",
      "OL Meget skal ske før en medalje kommer inden for rækkevidde. Dressurrytter Malene dsds har mistet troen på success",
      "Fodbold . Træner for Fjerritslev ser frem til sejr over Vordingborg. 'Det bliver en historisk kamp'",
      "Fodbold . De danske spillere skal op imod Sverige, som de tabte til i 2022",
      "Fodbold . De danske spillere vil forsøge at besejre Tyrkiet den kommende Lørdag i VM-kamp. Tyrkiet har aldriv været i en VM-finale",
      "Fodbold . De danske spillere tror på sejr mod Tyrkiet. 'Den skal vindes'",
      "Skisport . Sverige drømmer om flere medaljer og sejre til næste års OL . Træner forventer flere gode resultater",
      "Boksning . Kesler vil overraske alle og gøre det umulige. 'Jeg vinder i VM'",
      "Boksning . Kesler med stor selvtillid: 'Det bliver guld eller sølv til VM'",
    
     ]))


Results:
[[0.98257]
 [0.99697]
 [0.99998]
 [0.97882]
 [0.88391]]

 NON-Results:
[[0.57689]
 [0.00351]
 [0.14216]
 [0.5984 ]
 [0.03503]
 [0.05738]
 [0.01294]
 [0.42544]
 [0.97306]]


In [724]:
print("\n IN-BETWEEN-Results:") 
print(end_to_end_model.predict(
    [
       "Fodbold . Fjerritslev vandt i lørdags over Vordingborg 1-0. Den danske anfører dasdad dasdasd triumferer",
       "Fodbold . Fjerritslev vandt i lørdags over Vordingborg. Den danske anfører adasdasdd daddas triumferer",
       "Fodbold . Fjerritslev vandt i lørdags over Vordingborg. Efter kampen meddelyte den danske anfører sdfd sdfdf, at han skal under kniven",
       "Fodbold . Superliga-profil efter storsejr over Vordingborg. ' Den danske anfører fsdsdff sdffsd skal opereres og er ude i flere måneder",
       "Fodbold . Superliga-profil har meddelelse efter sejr. Den danske anfører fdfd sdffdf skal opereres og er ude i flere måneder",
       "Fodbold . Superliga-profil kan se frem til en længere pause. Den danske anfører fdfd sdfff skal opereres og er ude i flere måneder",
       
        ]))


 IN-BETWEEN-Results:
[[0.99998]
 [0.99986]
 [0.64692]
 [0.12974]
 [0.02857]
 [0.00005]]
